In [8]:
import subprocess
import os
print('Current conda environment:', os.environ['CONDA_DEFAULT_ENV'])
os.environ['TOKENIZERS_PARALLELISM'] = "false"

cwd = os.getcwd()
print(cwd)

import warnings
warnings.filterwarnings('ignore')

import random
random.seed(42)

Current conda environment: reinvent
/home/fts_g_ucla_edu/Projects/rips-relay/experiments


In [3]:
import pandas as pd

from rdkit import Chem

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, roc_auc_score, RocCurveDisplay

from xgboost import XGBClassifier

In [13]:
fragments = []

f = open("data/fragments.smi", "r")

for i in range(100):
    mol = f.readline()
    fragments.append(mol[:-1])

In [17]:
fragments[:5]

['C1c2cncnc2NC1',
 'Cc1n[nH]c(N)c1C#N',
 'NCc1cc(Br)cnc1',
 'CC(Nc1nccs1)=O',
 'Cc1c(CO)cncc1']

### Generate a distribution from each fragment for each model

In [19]:
reinvent_distributions, crem_distributions, coati_distributions, safe_distributions = [], [], [], []

In [21]:
for fragment in fragments:
    pass